# Simulations

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.sparse as spr
import networkx as nx
import spillover_effects as spef

## Functions

In [3]:
def sinv(A):
    """
    Find inverse of matrix A using numpy.linalg.solve
    Helpful for large matrices
    """
    if spr.issparse(A):
        n = A.shape[0]
        Ai = sp.linalg.spsolve(A.tocsc(), sp.eye(n, format='csc'))
    else:
        try:
            n = A.shape[0]
            Ai = np.linalg.solve(A, np.identity(n))
        except np.linalg.LinAlgError as err:
            if 'Singular matrix' in str(err):
                Ai = np.linalg.pinv(A)
            else:
                raise
    return Ai

In [4]:
def ball_vol(d,r):
    """
    Computes the volume of a d-dimensional ball of radius r. Used to construct RGG.

    Parameters
    ----------
    d : int 
        Dimension of space.
    r : float
        RGG parameter.
    """
    return np.pi**(d/2) * r**d / sp.special.gamma(d/2+1)

def gen_RGG(positions, r):
    """
    Generates an RGG.

    Parameters
    ----------
    positions : numpy array
        n x d array of d-dimensional positions, one for each of the n nodes.
    r : float
        RGG parameter.

    Returns
    -------
    RGG as NetworkX graph
    """
    kdtree = sp.spatial.cKDTree(positions)
    pairs = kdtree.query_pairs(r) # default is Euclidean norm
    RGG = nx.empty_graph(n=positions.shape[0], create_using=nx.Graph())
    RGG.add_edges_from(list(pairs))
    return RGG

## Misspecified Model

<!-- Potential outcomes follow linear-in-means model:

- Random position: $\quad \rho_i \sim \operatorname{Uniform}([0, 1]^2)$
- Random geometric graph with average degree $\bar{d}=2.5$: $\quad A \equiv \operatorname{RGG}(\rho, \bar{d})$ 
- Error with unobserved homophily: $\quad \varepsilon_i \sim \operatorname{N}(0, 0.5) + \alpha_{\rho}(\rho_{i,1} - 0.5)$
- Direct and spillover heterogeneous treatment effects: $\quad \beta^d_i \sim \operatorname{N}(1, 0.5)\ $ and $\ \beta^s_i \sim \operatorname{N}(0, 0.5)$
- Potential outcome under control exposure with network dependence: $\quad \tilde{Y}^0_i = \varepsilon_i + \sum_j \tilde{A}_{ij} \varepsilon_j$
- Potential outcome under direct exposure $D_i=1$: $\quad \tilde{Y}^d_i = \beta_i + \tilde{Y}^0_i$
- Potential outcome under spillover exposure $T_i=t$: $\quad \tilde{Y}^s_i = \sum_j \tilde{A}_{ij} \beta_j + \tilde{Y}^0_i$

Finite-population framework:

- Treatment indicator with direct propensity score $\pi^d_i=0.2$: $\quad D_i \sim \operatorname{Bernoulli}(\pi^d_i)$
- Exposure indicator: $\quad T_i \equiv \mathbf{1}\{\sum_j A_{ij} D_j > 0\}$
- Outcome: $\quad Y^*_i = $

Selection in a generalized Roy model:

- Selection error correlated with outcomes: $\quad \nu_i \sim N(0, 0.5) + \alpha_{\varepsilon}\varepsilon_i$
- Self-selection: $\quad S_i = \mathbf{1}\{(\alpha + \alpha_s \mathbf{1}\{t\} + \alpha_{\nu}\nu_i) > 0\}$
- If $\alpha_s$ is positive, the treatment induces individuals to participate in the outcome (Exposure Monotonicity) -->


In [5]:
# Parameters
rng = np.random.default_rng(seed=42)
nsim = 10000
n_list = [829, 1811, 2814] # 3, 8, and 14 schools
prob_treat = 0.25
avg_deg = 2.5
sigma_y, sigma_s, corr = 0.25, 0.5, 0.5
beta = 0.8


In [6]:
estimates_all = []
cvg_network_all = []
cvg_naive_all = []
cvg_oracle_all = []

for n in n_list:
    print(f"Sample size: {n}")
    estimates = pd.DataFrame(np.zeros((nsim, 5)),
                                columns=['Full Sample', 'Selected Sample', 'Lower Bound', 'Upper Bound', 'p-hat'])
    cvg_network = pd.DataFrame(np.zeros((nsim, 3)),
                            columns=['Full Sample', 'Selected Sample', 'Bounds'])
    cvg_naive = pd.DataFrame(np.zeros((nsim, 3)),
                            columns=['Full Sample', 'Selected Sample', 'Bounds'])
    cvg_oracle = pd.DataFrame(np.zeros((nsim, 3)),
                          columns=['Full Sample', 'Selected Sample', 'Bounds'])
    # Variables
    rho = rng.uniform(size=(n,2))
    A = gen_RGG(rho, (avg_deg/ball_vol(2,1)/n)**(1/2))
    errors = rng.multivariate_normal([0, 0], [[sigma_y**2, corr*sigma_y],
                                            [corr*sigma_y, sigma_s**2]], n)
    u = 0.5*errors[:,0] + 0.5*(rho[:,0] - 0.5)
    v = 0.5*errors[:,1] + 0.5*(rho[:,0] - 0.5)
    # Row-normalize A
    A_mat = nx.to_scipy_sparse_array(A, nodelist=range(n), format='csc')
    deg_seq = A_mat @ np.ones(n)
    r, c = A_mat.nonzero()
    rD_sp = spr.csr_matrix(((1.0/np.maximum(deg_seq,1))[r], (r,c)), shape=(A_mat.shape))
    A_norm = A_mat.multiply(rD_sp)
    lim_inv = spr.linalg.inv(spr.identity(n,format='csc') - beta*A_norm)
    weights, bw = spef.utils.kernel(A_mat)
    # Propensity score
    p0 = sp.stats.binom(deg_seq, prob_treat).pmf(0)
    # Outcomes
    Y = lim_inv @ u # (0*A_norm@D + 0*D + u)
    # Data
    df = pd.DataFrame({'Y': Y, 'p': 1-p0})

    for i in range(nsim):
        print(f"   Simulation {i+1}") if (i+1) % 1000 == 0 else None
        data = df.copy()
        # Direct and spillover treatment
        D = rng.binomial(1, prob_treat, n)
        T = ((A_mat @ D) > 0) * 1
        data['T'] = T
        # Observed outcome and selection
        S = lim_inv @ (0.1 + 0.2*A_norm@D + 0.2*D + v)
        sample = S > 0
        data['Ys'] = np.nan
        data.loc[sample, 'Ys'] = Y[sample]
        # Estimation - Full
        res = spef.WLS('Y', 'T', 'p', data, kernel_weights=weights, warn=False)
        estimates.iloc[i, 0] = res.summary.loc['spillover', 'coef']
        cvg_network.iloc[i, 0] = (res.summary.loc['spillover', 'p-val'] > 0.05) * 1
        res = spef.WLS('Y', 'T', 'p', data, warn=False)
        cvg_naive.iloc[i, 0] = (res.summary.loc['spillover', 'p-val'] > 0.05) * 1
        # Estimation - Selected
        res = spef.WLS('Ys', 'T', 'p', data, kernel_weights=weights, warn=False)
        estimates.iloc[i, 1] = res.summary.loc['spillover', 'coef']
        cvg_network.iloc[i, 1] = (res.summary.loc['spillover', 'p-val'] > 0.05) * 1
        res = spef.WLS('Ys', 'T', 'p', data, warn=False)
        cvg_naive.iloc[i, 1] = (res.summary.loc['spillover', 'p-val'] > 0.05) * 1
        # Estimation - Bounds
        res = spef.Bounds('Ys', 'T', 'p', data, kernel_weights=weights, warn=False)
        estimates.iloc[i, 2] = res.summary.loc['spillover', 'lower-bound']
        estimates.iloc[i, 3] = res.summary.loc['spillover', 'upper-bound']
        estimates.iloc[i, 4] = res.p_hat
        cvg_network.iloc[i, 2] = ((res.summary.loc['spillover', 'ci-low'] <= 0) & (res.summary.loc['spillover', 'ci-up'] >= 0)) * 1
        res = spef.Bounds('Ys', 'T', 'p', data, warn=False)
        cvg_naive.iloc[i, 2] = ((res.summary.loc['spillover', 'ci-low'] <= 0) & (res.summary.loc['spillover', 'ci-up'] >= 0)) * 1

    # Save oracle coverage
    cvg_oracle['Full Sample'] = (estimates['Full Sample'] - 1.96*estimates['Full Sample'].std() <= 0) & (estimates['Full Sample'] + 1.96*estimates['Full Sample'].std() >= 0)
    cvg_oracle['Selected Sample'] = (estimates['Selected Sample'] - 1.96*estimates['Selected Sample'].std() <= 0) & (estimates['Selected Sample'] + 1.96*estimates['Selected Sample'].std() >= 0)
    cvg_oracle['Bounds'] = (estimates['Lower Bound'] - 1.645*estimates['Lower Bound'].std() <= 0) & (estimates['Upper Bound'] + 1.645*estimates['Upper Bound'].std() >= 0)


    estimates['Sample Size'] = n
    estimates_all.append(estimates)
    cvg_network['Sample Size'] = n
    cvg_network_all.append(cvg_network)
    cvg_naive['Sample Size'] = n
    cvg_naive_all.append(cvg_naive)
    cvg_oracle['Sample Size'] = n
    cvg_oracle_all.append(cvg_oracle)

# pd.concat(estimates_all).to_csv('sim_results/estimates_misspecified.csv', index=False)
# pd.concat(cvg_network_all).to_csv('sim_results/cvg_network_misspecified.csv', index=False)
# pd.concat(cvg_naive_all).to_csv('sim_results/cvg_naive_misspecified.csv', index=False)
# pd.concat(cvg_oracle_all).to_csv('sim_results/cvg_oracle_misspecified.csv', index=False)

Sample size: 829
   Simulation 1000
   Simulation 2000
   Simulation 3000
   Simulation 4000
   Simulation 5000
   Simulation 6000
   Simulation 7000
   Simulation 8000
   Simulation 9000
   Simulation 10000
Sample size: 1811
   Simulation 1000
   Simulation 2000
   Simulation 3000
   Simulation 4000
   Simulation 5000
   Simulation 6000
   Simulation 7000
   Simulation 8000
   Simulation 9000
   Simulation 10000
Sample size: 2814
   Simulation 1000
   Simulation 2000
   Simulation 3000
   Simulation 4000
   Simulation 5000
   Simulation 6000
   Simulation 7000
   Simulation 8000
   Simulation 9000
   Simulation 10000


In [18]:
df_estimates = pd.read_csv('sim_results/estimates_misspecified.csv')
df_cvg_network = pd.read_csv('sim_results/cvg_network_misspecified.csv')
df_cvg_naive = pd.read_csv('sim_results/cvg_naive_misspecified.csv')
df_cvg_oracle = pd.read_csv('sim_results/cvg_oracle_misspecified.csv')

In [19]:
df_estimates.groupby('Sample Size').mean().round(2)

,Full Sample,Selected Sample,Lower Bound,Upper Bound,p-hat
Sample Size,,,,,
829,0.0,-0.19,-0.42,0.03,0.83
1811,-0.0,-0.19,-0.43,0.02,0.83
2814,-0.0,-0.16,-0.35,0.03,0.85


In [ ]:
df_cvg_oracle.groupby('Sample Size').mean().round(3)

,Full Sample,Selected Sample,Bounds
Sample Size,,,
829,0.952,0.399,0.975
1811,0.949,0.055,0.975
2814,0.953,0.064,0.983


In [23]:
df_cvg_network.groupby('Sample Size').mean().round(3)

,Full Sample,Selected Sample,Bounds
Sample Size,,,
829,0.950,0.435,0.949
1811,0.956,0.073,0.958
2814,0.956,0.079,0.965


In [21]:
df_cvg_naive.groupby('Sample Size').mean().round(3)

,Full Sample,Selected Sample,Bounds
Sample Size,,,
829,0.788,0.207,0.903
1811,0.809,0.019,0.922
2814,0.782,0.016,0.926


## Correctly Specified Model

<!-- Potential outcomes follow linear-in-means model:

- Random position: $\quad \rho_i \sim \operatorname{Uniform}([0, 1]^2)$
- Random geometric graph with average degree $\bar{d}=2.5$: $\quad A \equiv \operatorname{RGG}(\rho, \bar{d})$ 
- Error with unobserved homophily: $\quad \varepsilon_i \sim \operatorname{N}(0, 0.5) + \alpha_{\rho}(\rho_{i,1} - 0.5)$
- Direct and spillover heterogeneous treatment effects: $\quad \beta^d_i \sim \operatorname{N}(1, 0.5)\ $ and $\ \beta^s_i \sim \operatorname{N}(0, 0.5)$
- Potential outcome under control exposure with network dependence: $\quad \tilde{Y}^0_i = \varepsilon_i + \sum_j \tilde{A}_{ij} \varepsilon_j$
- Potential outcome under direct exposure $D_i=1$: $\quad \tilde{Y}^d_i = \beta_i + \tilde{Y}^0_i$
- Potential outcome under spillover exposure $T_i=t$: $\quad \tilde{Y}^s_i = \sum_j \tilde{A}_{ij} \beta_j + \tilde{Y}^0_i$

Finite-population framework:

- Treatment indicator with direct propensity score $\pi^d_i=0.2$: $\quad D_i \sim \operatorname{Bernoulli}(\pi^d_i)$
- Exposure indicator: $\quad T_i \equiv \mathbf{1}\{\sum_j A_{ij} D_j > 0\}$
- Outcome: $\quad Y^*_i = $

Selection in a generalized Roy model:

- Selection error correlated with outcomes: $\quad \nu_i \sim N(0, 0.5) + \alpha_{\varepsilon}\varepsilon_i$
- Self-selection: $\quad S_i = \mathbf{1}\{(\alpha + \alpha_s \mathbf{1}\{t\} + \alpha_{\nu}\nu_i) > 0\}$
- If $\alpha_s$ is positive, the treatment induces individuals to participate in the outcome (Exposure Monotonicity) -->


In [12]:
estimates_all = []
cvg_network_all = []
cvg_naive_all = []
cvg_oracle_all = []

for n in n_list:
    print(f"Sample size: {n}")
    estimates = pd.DataFrame(np.zeros((nsim, 5)),
                                columns=['Full Sample', 'Selected Sample', 'Lower Bound', 'Upper Bound', 'p-hat'])
    cvg_network = pd.DataFrame(np.zeros((nsim, 3)),
                            columns=['Full Sample', 'Selected Sample', 'Bounds'])
    cvg_naive = pd.DataFrame(np.zeros((nsim, 3)),
                            columns=['Full Sample', 'Selected Sample', 'Bounds'])
    cvg_oracle = pd.DataFrame(np.zeros((nsim, 3)),
                          columns=['Full Sample', 'Selected Sample', 'Bounds'])
    # Variables
    rho = rng.uniform(size=(n,2))
    A = gen_RGG(rho, (avg_deg/ball_vol(2,1)/n)**(1/2))
    errors = rng.multivariate_normal([0, 0], [[sigma_y**2, corr*sigma_y],
                                            [corr*sigma_y, sigma_s**2]], n)
    u = 0.5*errors[:,0] + 0.5*(rho[:,0] - 0.5)
    v = 0.5*errors[:,1] + 0.5*(rho[:,0] - 0.5)
    # Row-normalize A
    A_mat = nx.to_scipy_sparse_array(A, nodelist=range(n), format='csc')
    deg_seq = A_mat @ np.ones(n)
    r, c = A_mat.nonzero()
    rD_sp = spr.csr_matrix(((1.0/np.maximum(deg_seq,1))[r], (r,c)), shape=(A_mat.shape))
    A_norm = A_mat.multiply(rD_sp)
    lim_inv = spr.linalg.inv(spr.identity(n,format='csc') - beta*A_norm)
    weights, bw = spef.utils.kernel(A_mat)
    # Propensity score
    p0 = sp.stats.binom(deg_seq, prob_treat).pmf(0)
    # Outcomes
    Y = lim_inv @ u # (0*A_norm@D + 0*D + u)
    # Data
    df = pd.DataFrame({'Y': Y, 'p': 1-p0})

    for i in range(nsim):
        print(f"   Simulation {i+1}") if (i+1) % 1000 == 0 else None
        data = df.copy()
        # Direct and spillover treatment
        D = rng.binomial(1, prob_treat, n)
        T = ((A_mat @ D) > 0) * 1
        data['T'] = T
        # Observed outcome and selection
        S = 0.4 + 0.2*T + v
        sample = S > 0
        data['Ys'] = np.nan
        data.loc[sample, 'Ys'] = Y[sample]
        # Estimation - Full
        res = spef.WLS('Y', 'T', 'p', data, kernel_weights=weights, warn=False)
        estimates.iloc[i, 0] = res.summary.loc['spillover', 'coef']
        cvg_network.iloc[i, 0] = (res.summary.loc['spillover', 'p-val'] > 0.05) * 1
        res = spef.WLS('Y', 'T', 'p', data, warn=False)
        cvg_naive.iloc[i, 0] = (res.summary.loc['spillover', 'p-val'] > 0.05) * 1
        # Estimation - Selected
        res = spef.WLS('Ys', 'T', 'p', data, kernel_weights=weights, warn=False)
        estimates.iloc[i, 1] = res.summary.loc['spillover', 'coef']
        cvg_network.iloc[i, 1] = (res.summary.loc['spillover', 'p-val'] > 0.05) * 1
        res = spef.WLS('Ys', 'T', 'p', data, warn=False)
        cvg_naive.iloc[i, 1] = (res.summary.loc['spillover', 'p-val'] > 0.05) * 1
        # Estimation - Bounds
        res = spef.Bounds('Ys', 'T', 'p', data, kernel_weights=weights, warn=False)
        estimates.iloc[i, 2] = res.summary.loc['spillover', 'lower-bound']
        estimates.iloc[i, 3] = res.summary.loc['spillover', 'upper-bound']
        estimates.iloc[i, 4] = res.p_hat
        cvg_network.iloc[i, 2] = ((res.summary.loc['spillover', 'ci-low'] <= 0) & (res.summary.loc['spillover', 'ci-up'] >= 0)) * 1
        res = spef.Bounds('Ys', 'T', 'p', data, warn=False)
        cvg_naive.iloc[i, 2] = ((res.summary.loc['spillover', 'ci-low'] <= 0) & (res.summary.loc['spillover', 'ci-up'] >= 0)) * 1

    # Save oracle coverage
    cvg_oracle['Full Sample'] = (estimates['Full Sample'] - 1.96*estimates['Full Sample'].std() <= 0) & (estimates['Full Sample'] + 1.96*estimates['Full Sample'].std() >= 0)
    cvg_oracle['Selected Sample'] = (estimates['Selected Sample'] - 1.96*estimates['Selected Sample'].std() <= 0) & (estimates['Selected Sample'] + 1.96*estimates['Selected Sample'].std() >= 0)
    cvg_oracle['Bounds'] = (estimates['Lower Bound'] - 1.645*estimates['Lower Bound'].std() <= 0) & (estimates['Upper Bound'] + 1.645*estimates['Upper Bound'].std() >= 0)


    estimates['Sample Size'] = n
    estimates_all.append(estimates)
    cvg_network['Sample Size'] = n
    cvg_network_all.append(cvg_network)
    cvg_naive['Sample Size'] = n
    cvg_naive_all.append(cvg_naive)
    cvg_oracle['Sample Size'] = n
    cvg_oracle_all.append(cvg_oracle)

# pd.concat(estimates_all).to_csv('sim_results/estimates_specified.csv', index=False)
# pd.concat(cvg_network_all).to_csv('sim_results/cvg_network_specified.csv', index=False)
# pd.concat(cvg_naive_all).to_csv('sim_results/cvg_naive_specified.csv', index=False)
# pd.concat(cvg_oracle_all).to_csv('sim_results/cvg_oracle_specified.csv', index=False)

Sample size: 829
   Simulation 1000
   Simulation 2000
   Simulation 3000
   Simulation 4000
   Simulation 5000
   Simulation 6000
   Simulation 7000
   Simulation 8000
   Simulation 9000
   Simulation 10000
Sample size: 1811
   Simulation 1000
   Simulation 2000
   Simulation 3000
   Simulation 4000
   Simulation 5000
   Simulation 6000
   Simulation 7000
   Simulation 8000
   Simulation 9000
   Simulation 10000
Sample size: 2814
   Simulation 1000
   Simulation 2000
   Simulation 3000
   Simulation 4000
   Simulation 5000
   Simulation 6000
   Simulation 7000
   Simulation 8000
   Simulation 9000
   Simulation 10000


In [3]:
df_estimates = pd.read_csv('sim_results/estimates_specified.csv')
df_cvg_network = pd.read_csv('sim_results/cvg_network_specified.csv')
df_cvg_naive = pd.read_csv('sim_results/cvg_naive_specified.csv')
df_cvg_oracle = pd.read_csv('sim_results/cvg_oracle_specified.csv')

In [4]:
df_estimates.groupby('Sample Size').mean().round(2)

,Full Sample,Selected Sample,Lower Bound,Upper Bound,p-hat
Sample Size,,,,,
829,0.0,-0.06,-0.18,0.04,0.93
1811,0.0,-0.07,-0.16,0.03,0.94
2814,-0.0,-0.07,-0.17,0.03,0.93


In [5]:
df_cvg_oracle.groupby('Sample Size').mean().round(3)

,Full Sample,Selected Sample,Bounds
Sample Size,,,
829,0.950,0.888,0.979
1811,0.950,0.801,0.983
2814,0.949,0.690,0.984


In [6]:
df_cvg_network.groupby('Sample Size').mean().round(3)

,Full Sample,Selected Sample,Bounds
Sample Size,,,
829,0.954,0.895,0.970
1811,0.953,0.811,0.976
2814,0.954,0.713,0.979


In [7]:
df_cvg_naive.groupby('Sample Size').mean().round(3)

,Full Sample,Selected Sample,Bounds
Sample Size,,,
829,0.803,0.717,0.922
1811,0.796,0.579,0.934
2814,0.791,0.435,0.936
